In [1]:
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
class StockPrediction(tf.keras.Model):
  def __init__(self, max_seq_len, vocab_size, embed_size):
    super().__init__()
    self.max_seq_len = max_seq_len
    self.vocab_size = vocab_size
    self.embed_size = embed_size

    # 전 날의 기사를 인코딩. 길이가 정해져 있으므로, 양방향
    # 100은 그냥 별 생각 없이 넣었습니다.
    self.embed_layer = layers.Embedding(self.vocab_size, self.embed_size)
    self.news_encoder = layers.Bidirectional(layers.LSTM(100, return_sequences=True))

    self.stock_encoder = layers.LSTM(100, return_sequences=True)

    self.attention = layers.Attention()
    self.pooling = layers.GlobalAveragePooling1D()

    self.dense = layers.Dense(32, activation = 'relu')
    self.pred_layer = layers.Dense(1, activation = 'linear')

def call(self, inputs):
  news_inputs, stock_inputs = inputs

  embedded_news = self.embed_layer(news_inputs)
  news_seq_outs, lstm_out = self.news_encoder(embedded_news)

  stock_seq_outs, _ = self.stock_encoder(stock_inputs)

  attention = self.attention([news_seq_outs, stock_seq_outs])
  pooled = self.pooling(attention)

  out = self.dense(pooled)
  pred = self.pred_layer(out)

  return pred

In [8]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Counting objects: 100% (14762/14762), done.
remote: Compressing objects: 100% (13012/13012), done.
remote: Total 14763 (delta 1748), reused 14762 (delta 1748), pack-reused 1
Receiving objects: 100% (14763/14763), 56.19 MiB | 17.72 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (14737/14737), done.


In [9]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import keras

train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


In [10]:
train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [11]:
test

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [12]:
model = StockPrediction(max_seq_len=100, vocab_size=5000, embed_size=64)